In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

In [2]:
rating = pd.read_csv('/Users/blakemyers/Desktop/Jupyter/ml-1m/ratings.csv', sep = "::", error_bad_lines=False, encoding='latin-1', engine='python')

In [3]:
rating.rename({"1": "userId", "1193": "movieId", "5": "rating", "978300760": "timestamp"}, axis=1, inplace=True)

In [4]:
movie = pd.read_csv("/Users/blakemyers/Desktop/Jupyter/ml-1m/movies.csv", sep = "::", error_bad_lines=False, encoding='latin-1', engine='python')

In [5]:
movie.rename({"1": "movieId", "Toy Story (1995)": "title", "Animation|Children's|Comedy": "genre"}, axis=1, inplace=True)

In [6]:
movie_rating = pd.merge(rating, movie, on = 'movieId')

In [7]:
cols = ['timestamp']

In [8]:
movie_rating.drop(cols, axis=1, inplace=True)

In [9]:
numrate_movie = movie_rating.groupby("title")["rating"].count().reset_index()

In [10]:
numrate_movie.rename({"rating": "ratecount_movie"}, axis=1, inplace=True)

In [11]:
numrate_movie = numrate_movie.query("ratecount_movie >= 20")

In [12]:
ratings20plus = pd.merge(numrate_movie, movie_rating, on = 'title', how = 'inner')

In [13]:
numrate_user = ratings20plus.groupby("userId")["rating"].count().reset_index()

In [14]:
numrate_user.rename({"rating": "ratecount_user"}, axis=1, inplace=True)

In [15]:
numrate_user = numrate_user.query("ratecount_user >= 20")

In [16]:
ur20plus = pd.merge(ratings20plus, numrate_user, on = "userId", how = "inner")

In [17]:
scaler = MinMaxScaler()
ur20plus['rating'] = ur20plus['rating'].values.astype(float) #set rating values as float
rating_scaled = pd.DataFrame(scaler.fit_transform(ur20plus['rating'].values.reshape(-1,1)))
#place the rating values on a scale from -1 to 1
ur20plus['rating'] = rating_scaled

In [18]:
ur20plus = ur20plus.drop_duplicates(['userId', 'title']) #drop duplicates
user_movie_matrix = ur20plus.pivot(index='userId', columns='title', values='rating')
#create matrix (see below)
user_movie_matrix.fillna(0, inplace=True)

In [19]:
X_train, X_test = train_test_split(user_movie_matrix, train_size=0.8)
#split the training data (80%) from the testing data

In [20]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [21]:
num_input = ur20plus['title'].nunique()
# Deciding how many nodes each layer should have
n_nodes_inpl = num_input
n_nodes_hl1  = 256
n_nodes_outl = num_input  
# first hidden layer has num_input*32 weights and 32 biases
hidden_1_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_inpl+1,n_nodes_hl1]))}
# first hidden layer has 784*32 weights and 32 biases
output_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1+1,n_nodes_outl]))}

In [22]:
input_layer = tf.placeholder('float', [None, num_input])
# add a constant node to the first layer
# it needs to have the same shape as the input layer to be able to concatinate it later
input_layer_const = tf.fill( [tf.shape(input_layer)[0], 1] ,1.0  )
input_layer_concat =  tf.concat([input_layer, input_layer_const], 1)
# multiply output of input_layer wth a weight matrix 
layer_1 = tf.nn.sigmoid(tf.matmul(input_layer_concat,\
hidden_1_layer_vals['weights']))
# adding one bias node to the hidden layer
layer1_const = tf.fill( [tf.shape(layer_1)[0], 1] ,1.0  )
layer_concat =  tf.concat([layer_1, layer1_const], 1)
# multiply output of hidden with a weight matrix to get final output
output_layer = tf.matmul( layer_concat,output_layer_vals['weights'])
# output_true shall have the original shape for error calculations
output_true = tf.placeholder('float', [None, num_input])
# define cost function
meansq =    tf.reduce_mean(tf.square(output_layer - output_true))
# define optimizer
learn_rate = 0.1   # how fast the model should learn
optimizer = tf.train.AdagradOptimizer(learn_rate).minimize(meansq)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [23]:
# initialising variables and starting the session
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
# defining batch size, number of epochs and learning rate
batch_size = 100  # how many images to use together for training
hm_epochs =200    # how many times to go through the entire dataset
tot_images = X_train.shape[0] # total number of images

In [24]:
# running the model for a 200 epochs taking 100 users in batches
# total improvement is printed out after each epoch
for epoch in range(hm_epochs):
    epoch_loss = 0    # initializing loss (error) as 0
    
    for i in range(int(tot_images/batch_size)):
        epoch_x = X_train[ i*batch_size : (i+1)*batch_size ]
        _, c = sess.run([optimizer, meansq],\
               feed_dict={input_layer: epoch_x, \
               output_true: epoch_x})
        epoch_loss += c
        
    output_train = sess.run(output_layer,\
               feed_dict={input_layer:X_train})
    output_test = sess.run(output_layer,\
                   feed_dict={input_layer:X_test})
        
    if epoch <= 10 or epoch >= 190:
        print('MSE train', MSE(output_train, X_train),'MSE test', MSE(output_test, X_test))
        print('Epoch', epoch, '/', hm_epochs, 'loss:',epoch_loss)
    elif epoch == 11:
        print('.......................')
    else:
        continue

MSE train 46.78172693458639 MSE test 47.04147937731239
Epoch 0 / 200 loss: 3364.2936973571777
MSE train 30.856102592054057 MSE test 31.194315330043338
Epoch 1 / 200 loss: 1826.058198928833
MSE train 22.278783707306648 MSE test 22.655380074266667
Epoch 2 / 200 loss: 1264.5690650939941
MSE train 16.896151744216937 MSE test 17.282881246058494
Epoch 3 / 200 loss: 936.4319343566895
MSE train 13.306052216637273 MSE test 13.681309024871702
Epoch 4 / 200 loss: 723.3825130462646
MSE train 10.886063483586833 MSE test 11.213885796187139
Epoch 5 / 200 loss: 579.7373304367065
MSE train 9.198073524130821 MSE test 9.483610730197759
Epoch 6 / 200 loss: 482.072566986084
MSE train 7.968654641045418 MSE test 8.227080432288938
Epoch 7 / 200 loss: 412.48894214630127
MSE train 7.030609559204108 MSE test 7.266145205645336
Epoch 8 / 200 loss: 360.6009135246277
MSE train 6.296513058310974 MSE test 6.512188913212953
Epoch 9 / 200 loss: 320.5431351661682
MSE train 5.705229723754047 MSE test 5.904772754317141
Epo

In [25]:
# a top 10 ranking for a new user (9999991) who rates "Chariots of Fire (1981)" with a 5
ur20plus = ur20plus.append(pd.DataFrame([["Chariots of Fire (1981)",1,9999991,1,5,"genre",1]], columns =ur20plus.columns), ignore_index=True)
scaler = MinMaxScaler()
ur20plus['rating'] = ur20plus['rating'].values.astype(float)
rating_scaled = pd.DataFrame(scaler.fit_transform(ur20plus['rating'].values.reshape(-1,1)))
ur20plus['rating'] = rating_scaled
ur20plus = ur20plus.drop_duplicates(['userId', 'title'])
user_movie_matrix = ur20plus.pivot(index='userId', columns='title', values='rating')
user_movie_matrix.fillna(0, inplace=True)
pred_data = pd.DataFrame()
preds = sess.run(output_layer, feed_dict={input_layer: user_movie_matrix})
pred_data = pred_data.append(pd.DataFrame(preds))
pred_data = pred_data.stack().reset_index(name='rating')
pred_data.columns = ['userId', 'title', 'rating']
users = user_movie_matrix.index.tolist()
movies = user_movie_matrix.columns.tolist()
pred_data['userId'] = pred_data['userId'].map(lambda value: users[value])
pred_data['title'] = pred_data['title'].map(lambda value: movies[value])
keys = ['userId', 'title']
index_1 = pred_data.set_index(keys).index
index_2 = ur20plus.set_index(keys).index
top_ten_ranked = pred_data[~index_1.isin(index_2)]
top_ten_ranked = top_ten_ranked.sort_values(['userId', 'rating'], ascending=[True, False])
top_ten_ranked = top_ten_ranked.groupby('userId').head(10)
top_ten_ranked.loc[top_ten_ranked['userId'] == 9999991]

,userId,title,rating
18299163,9999991,Koyaanisqatsi (1983),2.801270
18297998,9999991,Bloodsport 2 (1995),2.791837
18298777,9999991,Gone with the Wind (1939),2.693222
18299597,9999991,Nikita (La Femme Nikita) (1990),2.602829
18297943,9999991,"Bicycle Thief, The (Ladri di biciclette) (1948)",2.488431
18300429,9999991,Touch (1997),2.420003
18300070,9999991,"Shadow, The (1994)",2.386274
18297806,9999991,Armageddon (1998),2.374817
18300308,9999991,Switchblade Sisters (1975),2.362792
18300362,9999991,Things Change (1988),2.353522


In [26]:
# a top 10 ranking for a new user (9999992) who rates "Hurricane, The (1999)" with a 3
ur20plus = ur20plus.append(pd.DataFrame([["Hurricane, The (1999)",1,9999992,1,3,"genre",1]], columns =ur20plus.columns), ignore_index=True)
scaler = MinMaxScaler()
ur20plus['rating'] = ur20plus['rating'].values.astype(float)
rating_scaled = pd.DataFrame(scaler.fit_transform(ur20plus['rating'].values.reshape(-1,1)))
ur20plus['rating'] = rating_scaled
ur20plus = ur20plus.drop_duplicates(['userId', 'title'])
user_movie_matrix = ur20plus.pivot(index='userId', columns='title', values='rating')
user_movie_matrix.fillna(0, inplace=True)
pred_data = pd.DataFrame()
preds = sess.run(output_layer, feed_dict={input_layer: user_movie_matrix})
pred_data = pred_data.append(pd.DataFrame(preds))
pred_data = pred_data.stack().reset_index(name='rating')
pred_data.columns = ['userId', 'title', 'rating']
users = user_movie_matrix.index.tolist()
movies = user_movie_matrix.columns.tolist()
pred_data['userId'] = pred_data['userId'].map(lambda value: users[value])
pred_data['title'] = pred_data['title'].map(lambda value: movies[value])
keys = ['userId', 'title']
index_1 = pred_data.set_index(keys).index
index_2 = ur20plus.set_index(keys).index
top_ten_ranked = pred_data[~index_1.isin(index_2)]
top_ten_ranked = top_ten_ranked.sort_values(['userId', 'rating'], ascending=[True, False])
top_ten_ranked = top_ten_ranked.groupby('userId').head(10)
top_ten_ranked.loc[top_ten_ranked['userId'] == 9999992]

,userId,title,rating
18300922,9999992,"Basketball Diaries, The (1995)",2.588409
18302704,9999992,Open Your Eyes (Abre los ojos) (1997),2.289147
18303396,9999992,They Might Be Giants (1971),2.018482
18302205,9999992,Koyaanisqatsi (1983),2.006338
18301758,9999992,George of the Jungle (1997),1.930323
18303519,9999992,Two Girls and a Guy (1997),1.921138
18303360,9999992,Tall Tale (1994),1.909445
18303179,9999992,"Slingshot, The (Kådisbellan ) (1993)",1.907789
18301434,9999992,"Devil's Own, The (1997)",1.904878
18302806,9999992,Pink Flamingos (1972),1.900558
